
[Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk) Kaggle 上最近的结构化数据项目，上手实践一下。竞赛目标是预测贷款申请人是否有违约行为。

因为涉及到多个表的处理，所以很多参赛者使用了 Featuretools 工具自动化进行特征工程，
我主要参考了 Will Koehrsen 这位作者写了一系列教程，主要从 简介 -> 手工特征工程-> 利用库来进行自动特征工程。

- 简介
[Start Here: A Gentle Introduction
](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction)
- 手工特征工程
[Introduction to Manual Feature Engineering
](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering)
- 手工特征工程2
[Introduction to Manual Feature Engineering P2](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2
)
- 自动特征工程(基础)
[Automated Feature Engineering Basics
](https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics/notebook)
- 自动特征工程(探索)
[Tuning Automated Feature Engineering (Exploratory)
](https://www.kaggle.com/willkoehrsen/tuning-automated-feature-engineering-exploratory/notebook)

这篇notebook，我直接介绍Featuretools如何上手。

主要使用竞赛数据介绍Featuretools的使用场景，实践代码则尽量选取官方介绍中的代码，我觉得官方数据不太好用的时候，会生成一些数据。

## 1. 竞赛数据简介

**数据聚合方式**

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

application_{train|test}.csv 是主表，包含label和一些特征，其他表都要关联到这两个表上。

另外的两个数据来源，左侧的两个是来自信用卡管理机构的信息，右侧的是来自本公司之前的数据。

bureau.csv 向上关联 application_train/test.csv 是根据 `SK_ID_DURR` 链接
bureau.csv 向下关联 bureau_balance.csv 是根据 `SK_ID_BUREAU` 链接

`SK_ID_DURR` -> `SK_ID_BUREAU` 是一对多的关系，也就是当前每条申请记录在bureau.csv 中可能有多个账户
`SK_ID_BUREAU` 中每个账户也每个月都有信息
所以，将新表融合进我们主表的过程，是从下向上不断 `分组 -> 聚合 -> 链接` 的过程
![](https://ws2.sinaimg.cn/large/006tKfTcgy1fshmi5yejuj30qf0ad3zk.jpg)

对应左边本公司数据也是一样。

**针对不同类型数据进行不同的聚合**

```
客户之前的账户总数 -> 计数信息
数值型 -> min max sum median
分类型 -> 各分类的个数 占比
```

- 数值型实例生成的特征
![](https://ws1.sinaimg.cn/large/006tKfTcgy1fshpobksesj30k205eq3a.jpg)

- 分类型特征生成过程和结果

![](https://ws1.sinaimg.cn/large/006tKfTcgy1fshppq0gnqj30a80ks0tg.jpg)
![](https://ws2.sinaimg.cn/large/006tKfTcgy1fshpqzyl5xj31b408sdgr.jpg)

##  2.概念介绍

#### 2.1 特征深度综合 Deep Feature Synthesis

看到自动特征工程这种描述，脑子里总是忍不住幻想，有个开源库，我把数据扔进去，数据直接填充好，清洗干净，生成了足够的特征，而且做过了筛选。。。。。。

当然没这种好事，featuretool这个库主要使用场景是数据源的表比较多，当然就是这次竞赛的场景。作用主要在自动化的进行**特征深度综合**。

![](https://ws3.sinaimg.cn/large/006tNc79gy1fsiukivet2j315p0ex412.jpg)

我们在之前已经实现了纯手工`分组 -> 聚合 -> 链接`过程，这是一张官方blog给出的介绍用图，这种聚合过程可以看做是浅层的特征聚合。

![](https://ws4.sinaimg.cn/large/006tNc79gy1fsiv7peuvxj30sa0b7q4n.jpg)

当表的关系比较复杂，有多层id关系的时候，例如在网购场景中 `用户id -> 订单id -> 商品id` 这种每层都是一对多的表关系。图中生成的特征含义是：*每位客户，订单中所有商品总价的均值*

因为每层的聚合关系都有多种，导致了生成特征的数量随着聚合层数成指数级增长。

表链接在pandas 或者 SQL 中都是有对应操作的，featuretools的主要功能是使整个过程自动化，只需要控制聚合函数的选择，大大减少了手工作业，增加了特征的数量。


以下是官方的博客简介和最早分享的论文
- [Deep Feature Synthesis: How Automated Feature Engineering Works](https://www.featurelabs.com/blog/deep-feature-synthesis/)
- [Deep Feature Synthesis:Towards Automating Data Science Endeavors](https://www.featurelabs.com/wp-content/uploads/2017/12/DSAA_DSM_2015-1.pdf)

#### 2.2 实体 和 实体集合 Entities and EntitySets

**实体集合** 可以看做是一系列的表和他们组成关系构成的对象

构建实体集合 和 在实体集合中添加实体(dataframe)的基本操作

In [1]:
import pandas as pd
import warnings
# 一些警告设定不显示，主要是操作过时的类型
warnings.filterwarnings('ignore')

import featuretools as ft


# featuretools 自带的数据生成工具生成的数据，也是官方文档实例数据 
# 我觉得入门还是有点复杂，我做了简化，只生成了两个表，特征生成的过程看的更清除一点
# 表的基本关系如下  
# customer(消费者) -> transaction(交易)
# 每个消费者有多次交易

customers_df = pd.read_csv('./data/customers.csv') 
transactions_df = pd.read_csv('./data/transactions.csv')

In [2]:
customers_df

,customer_id,zip_code,join_date
0,1,60091,2008/1/1
1,2,2139,2008/2/20


In [3]:
transactions_df

,transaction_id,transaction_time,amount,customer_id
0,1,2014/1/1 0:00,107.00,1
1,4,2014/1/1 0:01,12.55,1
2,2,2014/1/1 0:02,12.97,1
3,3,2014/1/1 0:03,11.59,2
4,5,2014/1/1 0:04,37.19,2
5,6,2014/1/1 0:05,91.15,2


两个表通过 customer_id 相连接

In [4]:
# 构建 实体集合对象
es = ft.EntitySet(id="transactions")

# 在实体集合对象中添加实体(dataframe)
es = es.entity_from_dataframe(entity_id="transactions",
                              dataframe=transactions_df,
                              index="transaction_id",
                              time_index="transaction_time")

es = es.entity_from_dataframe(entity_id="customers",
                              dataframe=customers_df,
                              index="customer_id")

#### 2.3 表关系 Relationship
表关系的概念也存在于各种关系型数据库之中，对于一对多最好的类比就是 `父母(parent) -> 孩子(child)`
注意不要有菱形图关系，也就是一个孩子存在多个父母的关系，以防止聚合时候的冲突。

构建表关系 在实体集合中添加表关系的基本操作：

In [5]:
# 构建表关系对象 
new_relationship = ft.Relationship(es["customers"]["customer_id"],
                                   es["transactions"]["customer_id"])

# 将关系对象添加到 实体集合中
es = es.add_relationship(new_relationship) 

#### 2.4 特征基本操作 Feature primitives

特征基本操作，之前提到的聚合过程归属于特征基本操作，在Featuretools中将特征聚合和特征变换统一归类为特征基本操作 Feature primitives

- 统计聚合 Aggregation
- 变换 Transformation

常用的操作：
![](https://ws3.sinaimg.cn/large/006tNc79gy1fsiwlzv2ipj30ip09r75g.jpg)

![](https://ws4.sinaimg.cn/large/006tNc79gy1fsiwmgmjqdj30mi09kt9z.jpg)

#### 2.5 通过深度特征综合生成数据

In [6]:
# dfs 就是 特征深度综合 Deep Feature Synthesis 的缩写
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'customers',
                                       trans_primitives = ['year', 'month', 'week', 'day'],
                                       agg_primitives=['mean', 'max', 'min'],
                                       max_depth = 3, features_only=False, verbose = True)

Built 8 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


In [7]:
feature_matrix

,zip_code,MEAN(transactions.amount),MAX(transactions.amount),MIN(transactions.amount),YEAR(join_date),MONTH(join_date),WEEK(join_date),DAY(join_date)
customer_id,,,,,,,,
1,60091,44.173333,107.00,12.55,2008,1,1,1
2,2139,46.643333,91.15,11.59,2008,2,8,20


产生了如下变化

- zip_code 保留
- join_date 拆解成了 年 月 星期 日 信息 
- transactions.amount 聚合的 meam max min 信息添加进了 customers表中

值得注意的是，transactions的时间信息没有整合进来，本来以为时间特征也能进行个mean聚合啥的



### 总结：

使用 Featuretools 的基本流程为：

- 构建 实体集合
- 实体集合添加 实体(表)
- 实体集合添加 表关系
- 深度特征综合生成新特征

第一篇先写到这里，后续将也许更新下面的内容。。 

- 时间特征的处理
- 根据先验知识做一些特征设定 seed features
- 根据分类特征的某一类进行统计聚合，“interesting” values

主要参考官方文档：
[automated_feature_engineering](https://docs.featuretools.com/automated_feature_engineering/afe.html)